# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [48]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys1 import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [49]:
#Read csv file 
csvpath="/Users/apple/Desktop/bootcamp/6-Python-APIs/Python-api-Challenge/starter_code/cities.csv"
Vacation_data=pd.read_csv(csvpath)
Vacation_data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,North Bend,1,US,1584037641,61,43.41,-124.22,48.992,4.10
1,1,Hambantota,51,LK,1584037896,100,6.12,81.12,77.000,1.50
2,2,Mīzan Teferī,59,ET,1584037897,70,6.98,35.58,69.620,1.76
3,3,Khovd,0,MN,1584037874,43,48.01,91.64,12.020,0.85
4,4,Mgandu,84,TZ,1584037900,96,-5.95,34.13,65.120,1.77


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [50]:
# Create heat layer
locations=Vacation_data[["Lat","Lng"]]
humidity=Vacation_data["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:

#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
Perfect_weather_cities=Vacation_data[(Vacation_data["Max Temp"]>70)&(Vacation_data["Max Temp"]<80)&(Vacation_data["Wind Speed"]<10)&(Vacation_data["Cloudiness"]==0)]
Perfect_weather_cities.dropna()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
88,88,Matara,0,LK,1584037995,81,5.95,80.54,75.56,3.56
128,128,Broken Hill,0,AU,1584037916,49,-31.95,141.43,73.40,5.10
159,159,Mount Isa,0,AU,1584037841,83,-20.73,139.50,71.60,2.60
181,181,Kununurra,0,AU,1584037941,88,-15.77,128.73,75.20,0.88
192,192,Yulara,0,AU,1584038134,25,-25.24,130.99,77.00,3.60
256,256,São Filipe,0,CV,1584038211,58,14.90,-24.50,77.54,4.93
266,266,Inhambane,0,MZ,1584038056,81,-23.86,35.38,78.08,5.42
272,272,Namuac,0,PH,1584038236,90,18.61,121.17,70.52,1.28
367,367,Vallenar,0,CL,1584038349,43,-28.57,-70.76,76.64,5.34
420,420,Tura,0,IN,1584038413,44,25.52,90.22,73.94,1.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [52]:
#Create DataFrame hotel_df
hotel_df=Perfect_weather_cities.dropna()
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
search_range=5000
search_type="hotel"
search_detail={
    "keyword":search_type,
    "radius":search_range,
    "key":g_key
}
#Define empty column and lists to store hotel name
hotel_df["Hotel Name"]=""
hotel_list=[]
city_list=[]
country_list=[]
#Repeated API requests
for index,row in hotel_df.iterrows():
    try:
        search_detail["location"]=f"{row['Lat']},{row['Lng']}"
        result=requests.get(base_url,params=search_detail)
        results=result.json()
        hotel_list.append(results["results"][0]["name"])
        city_list.append(row['City'])
        country_list.append(row['Country'])
    except IndexError:
        hotel_list.append("no hotel found")
        city_list.append(row['City'])
        country_list.append(row['Country'])
#Load hotel name into column
hotel_df["Hotel Name"]=hotel_list
hotel_df.head()

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
88,88,Matara,0,LK,1584037995,81,5.95,80.54,75.56,3.56,ParrotFish Bay
128,128,Broken Hill,0,AU,1584037916,49,-31.95,141.43,73.40,5.10,Duke of Cornwall Inn
159,159,Mount Isa,0,AU,1584037841,83,-20.73,139.50,71.60,2.60,Abacus Motel
181,181,Kununurra,0,AU,1584037941,88,-15.77,128.73,75.20,0.88,Hotel Kununurra
192,192,Yulara,0,AU,1584038134,25,-25.24,130.99,77.00,3.60,Sails in the Desert


In [47]:
#Define coordinate for perfect weather city
cities_coordinates=hotel_df[["Lat","Lng"]]

# Add marker layer ontop of heat map
label=[[hotel_list[i], city_list[i],country_list[i]] for i in range(len(hotel_list))]
markers=gmaps.marker_layer(cities_coordinates,info_box_content=[f"Name:{j[0]},City:{j[1]},Country:{j[2]}" for j in label])
fig.add_layer(markers)
#Display map
fig


Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row)_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]